In [18]:
from pprint import pprint

from modelconstruct import Mexplode
from modelpattern import list_extract

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
eq = '''
list ages = ages : age_0 * age_11 
list sexes = sexes   : female male /
             fertile :     1     0
£ primo population is (ultimo)population in age group before
doable [ages ages_nostart]  pop_primo__a = pop__ab(-1)

£ for first age group the population depends on all births 
doable [ages ages_start]   pop_before_death__a   = birth__total__{sexes} + migration__a
doable [ages ages_nostart] pop_before_death__a   = pop_primo__a + migration__a 

£ --- Step 3: Deaths ---
doable  [ages ages_end ]  death_rate__a =1.0 
doable  <sum=total>          death__a = pop_before_death__a  * death_rate__a

£ --- Step 4: End-of-period population ---
doable  <sum=total>          pop__a  = pop_before_death__a  - death__a 

£ --- Step 5: Births from fertile ages (all except AGE_0) ---
do ages ages_nostart = 1
   do    sexes fertile = 1
        birth__a = pop_primo__a * fertility__a
   enddo
enddo
birth_total = sum(ages ages_nostart=1,sum(sexes fertile=1,birth__a))        

birth__total__female = FRAC_BIRTH__female * birth_total
birth__total__male = (1 - FRAC_BIRTH__female) * birth_total


£ --- Step 6: Totals and consistency check ---
migration__total = sum(ages,sum(sexes,migration__a))
pop__total_check  = (pop__total(-1) + birth__total + migration__total - death__total) - pop__total


'''
print('This Input:')
(mm := Mexplode(eq,
replacements=[('__ab','__{sexes}__{ages_before}'),('__a','__{sexes}__{ages}')]
               
               )).showclean_frml_statements  
print('\nCreates this Output')
mm.show

This Input:
Clean_frml_statements: 
LIST AGES =
    AGES : AGE_0 * AGE_11  $
LIST SEXES =
    SEXES   : FEMALE MALE /
    FERTILE : 1      0  $
£ PRIMO POPULATION IS (ULTIMO)POPULATION IN AGE GROUP BEFORE
DOABLE [AGES AGES_NOSTART]  POP_PRIMO__{SEXES}__{AGES} = POP__{SEXES}__{AGES_BEFORE}(-1)$

£ FOR FIRST AGE GROUP THE POPULATION DEPENDS ON ALL BIRTHS 
DOABLE [AGES AGES_START]   POP_BEFORE_DEATH__{SEXES}__{AGES}   = BIRTH__TOTAL__{SEXES} + MIGRATION__{SEXES}__{AGES}$
DOABLE [AGES AGES_NOSTART] POP_BEFORE_DEATH__{SEXES}__{AGES}   = POP_PRIMO__{SEXES}__{AGES} + MIGRATION__{SEXES}__{AGES} $

£ --- STEP 3: DEATHS ---
DOABLE  [AGES AGES_END ]  DEATH_RATE__{SEXES}__{AGES} =1.0 $
DOABLE  <SUM=TOTAL>          DEATH__{SEXES}__{AGES} = POP_BEFORE_DEATH__{SEXES}__{AGES}  * DEATH_RATE__{SEXES}__{AGES}$

£ --- STEP 4: END-OF-PERIOD POPULATION ---
DOABLE  <SUM=TOTAL>          POP__{SEXES}__{AGES}  = POP_BEFORE_DEATH__{SEXES}__{AGES}  - DEATH__{SEXES}__{AGES} $

£ --- STEP 5: BIRTHS FROM FERTILE AGE

In [21]:
mm.mmodel.pop__age_2.tracepre(up=6,size=(10,10))

AttributeError: The specification:"pop__age_2" did not match a method, property or variable name

In [ ]:
import pandas as pd

# --- model structure ---
ages = [f'AGE_{i}' for i in range(11)]  # AGE_0 ... AGE_10
variables = [f'POP__{age}' for age in ages]

years = range(2010, 2031)  # example period (2010–2030)

# --- initial population distribution (illustrative numbers, can be replaced) ---
# e.g. a pyramid decreasing by age
base_population = {
    f'POP__AGE_0':  400000,
    f'POP__AGE_1':  390000,
    f'POP__AGE_2':  385000,
    f'POP__AGE_3':  380000,
    f'POP__AGE_4':  370000,
    f'POP__AGE_5':  360000,
    f'POP__AGE_6':  340000,
    f'POP__AGE_7':  320000,
    f'POP__AGE_8':  300000,
    f'POP__AGE_9':  270000,
    f'POP__AGE_10': 240000,
}

# --- create the DataFrame ---
df = pd.DataFrame(index=years, columns=variables, dtype=float)

# fill only the first year (e.g. 2010)
start_year = 2010
for var, value in base_population.items():
    df.loc[start_year, var] = value

# optional: forward-fill or leave NaN for model simulation
df = df.ffill()

df.head()


In [ ]:
import pandas as pd
import numpy as np

years = range(2010, 2031)  # same period as your population DataFrame
ages = [f'AGE_{i}' for i in range(11)]

# --------------------
# 1. DEATH RATES (by age)
# --------------------
death_rates = {
    'AGE_0': 0.010,  # infant mortality
    'AGE_1': 0.001,
    'AGE_2': 0.0008,
    'AGE_3': 0.0007,
    'AGE_4': 0.0008,
    'AGE_5': 0.0010,
    'AGE_6': 0.0020,
    'AGE_7': 0.0040,
    'AGE_8': 0.0100,
    'AGE_9': 0.0200,
    # AGE_10 assumed handled in model (set = 1.0)
}

death_rate_vars = [f'DEATH_RATE__{age}' for age in ages[:-1]]  # AGE_0–AGE_9
df_death = pd.DataFrame(index=years, columns=death_rate_vars, dtype=float)
for var in death_rate_vars:
    df_death[var] = death_rates[var.split('__')[-1]]

# --------------------
# 2. FERTILITY RATES (fertile ages only)
# --------------------
fertility_rates = {
    'AGE_1': 0.000,  # not fertile
    'AGE_2': 0.010,
    'AGE_3': 0.040,
    'AGE_4': 0.070,
    'AGE_5': 0.080,
    'AGE_6': 0.060,
    'AGE_7': 0.030,
    'AGE_8': 0.010,
    'AGE_9': 0.002,
    'AGE_10': 0.000,
}

fertility_vars = [f'FERTILITY__{age}' for age in ages[1:]]  # AGE_1–AGE_10
df_fert = pd.DataFrame(index=years, columns=fertility_vars, dtype=float)
for var in fertility_vars:
    df_fert[var] = fertility_rates[var.split('__')[-1]]

# --------------------
# 3. MIGRATION (net migration, can be + or −)
# --------------------
migration_pattern = {
    'AGE_0':  500,   # babies moving with parents
    'AGE_1':  400,
    'AGE_2':  200,
    'AGE_3':  100,
    'AGE_4':   50,
    'AGE_5':  -50,
    'AGE_6': -100,
    'AGE_7': -200,
    'AGE_8': -300,
    'AGE_9': -400,
    'AGE_10': -500,
}

migration_vars = [f'MIGRATION__{age}' for age in ages]
df_mig = pd.DataFrame(index=years, columns=migration_vars, dtype=float)
for var in migration_vars:
    df_mig[var] = migration_pattern[var.split('__')[-1]]

# --------------------
# 4. Combine all
# --------------------
df_exo = pd.concat([df_death, df_fert, df_mig], axis=1)
df_exo.index.name = 'year'

df_exo.head()


In [ ]:
df_full = pd.concat([df, df_exo], axis=1)
df_full

In [ ]:
mpop = mm.mmodel
mpop(df_full)

In [ ]:
with mpop.set_smpl(2010,2030):
    print(mpop['pop__total death__total birth__total'].df)